# Deep Learning Module: Neural Networks
Goal: Create a multi-layer perceptron neural network model to predict on a labeled dataset of your choosing. Compare this model to either a boosted tree or a random forest model and describe the relative tradeoffs between complexity and accuracy. Vary the hyperparameters of our MLP!

# Data Set Description:
    This folder contains data behind the story Comic Books Are Still Made By Men, For Men And About Men.

The data comes from Marvel Wikia and DC Wikia. Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: dc-wikia-data.csv and marvel-wikia-data.csv. Each file has the following variables:

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

import os
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm

import re
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Import Perceptron.
from sklearn.linear_model import Perceptron

In [3]:
dc_hero_df = pd.read_csv('/Users/mehrunisaqayyum/Downloads/dc-wikia-data.csv')
dc_hero_df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,66302,Nadine West (New Earth),\/wiki\/Nadine_West_(New_Earth),Public Identity,Good Characters,NaN,NaN,Female Characters,NaN,Living Characters,NaN,NaN,NaN
6892,283475,Warren Harding (New Earth),\/wiki\/Warren_Harding_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN
6893,283478,William Harrison (New Earth),\/wiki\/William_Harrison_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN
6894,283471,William McKinley (New Earth),\/wiki\/William_McKinley_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


In [4]:
marvel_hero_df = pd.read_csv('/Users/mehrunisaqayyum/Downloads/marvel-wikia-data.csv')
marvel_hero_df

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


In [9]:
dc_hero_df.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'YEAR'],
      dtype='object')

In [10]:
marvel_hero_df.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'Year'],
      dtype='object')

## Exploratory Data Analysis: Data Cleaning

In [11]:
#Fill in Missing Values for Selected Variables for DC and Marvel data sets
dc_hero_df['ID'] = dc_hero_df['ID'].fillna(dc_hero_df['ID'].mode()[0])
dc_hero_df['ALIGN'] = dc_hero_df['ALIGN'].fillna(dc_hero_df['ALIGN'].mode()[0])
dc_hero_df['EYE'].fillna(dc_hero_df['EYE'].mode()[0], inplace = True)
dc_hero_df['HAIR'].fillna(dc_hero_df['HAIR'].mode()[0], inplace = True)
dc_hero_df['SEX'].fillna(dc_hero_df['SEX'].mode()[0], inplace = True)
dc_hero_df['ALIVE'].fillna(dc_hero_df['ALIVE'].mode()[0], inplace = True)
dc_hero_df['APPEARANCES'].fillna(dc_hero_df['APPEARANCES'].mode()[0], inplace = True)
dc_hero_df['FIRST APPEARANCE'].fillna(dc_hero_df['FIRST APPEARANCE'].mode()[0], inplace = True)
dc_hero_df['YEAR'].fillna(dc_hero_df['YEAR'].mode()[0], inplace = True)

#Second data set
marvel_hero_df['ID'] = marvel_hero_df['ID'].fillna(marvel_hero_df['ID'].mode()[0])
marvel_hero_df['ALIGN'] = marvel_hero_df['ALIGN'].fillna(marvel_hero_df['ALIGN'].mode()[0])
marvel_hero_df['EYE'].fillna(marvel_hero_df['EYE'].mode()[0], inplace = True)
marvel_hero_df['HAIR'].fillna(marvel_hero_df['HAIR'].mode()[0], inplace = True)
marvel_hero_df['SEX'].fillna(marvel_hero_df['SEX'].mode()[0], inplace = True)
marvel_hero_df['ALIVE'].fillna(marvel_hero_df['ALIVE'].mode()[0], inplace = True)
marvel_hero_df['APPEARANCES'].fillna(marvel_hero_df['APPEARANCES'].mode()[0], inplace = True)
marvel_hero_df['FIRST APPEARANCE'].fillna(marvel_hero_df['FIRST APPEARANCE'].mode()[0], inplace = True)
marvel_hero_df['Year'].fillna(marvel_hero_df['Year'].mode()[0], inplace = True)

### Merge Data Sets

#merged_df = pd.merge(dc_hero_df, marvel_hero_df, left_on='ID', right_on='ID')
#merged_df

In [72]:
#pd.concat.marvel_hero_df dc
new_df = pd.concat([marvel_hero_df, dc_hero_df], axis=0)
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,ALIGN,ALIVE,APPEARANCES,EYE,FIRST APPEARANCE,GSM,HAIR,ID,SEX,YEAR,Year,name,page_id,urlslug
0,Good Characters,Living Characters,4043.0,Hazel Eyes,Aug-62,NaN,Brown Hair,Secret Identity,Male Characters,NaN,1962.0,Spider-Man (Peter Parker),1678,\/Spider-Man_(Peter_Parker)
1,Good Characters,Living Characters,3360.0,Blue Eyes,Mar-41,NaN,White Hair,Public Identity,Male Characters,NaN,1941.0,Captain America (Steven Rogers),7139,\/Captain_America_(Steven_Rogers)
2,Neutral Characters,Living Characters,3061.0,Blue Eyes,Oct-74,NaN,Black Hair,Public Identity,Male Characters,NaN,1974.0,"Wolverine (James \""Logan\"" Howlett)",64786,\/Wolverine_(James_%22Logan%22_Howlett)
3,Good Characters,Living Characters,2961.0,Blue Eyes,Mar-63,NaN,Black Hair,Public Identity,Male Characters,NaN,1963.0,"Iron Man (Anthony \""Tony\"" Stark)",1868,\/Iron_Man_(Anthony_%22Tony%22_Stark)
4,Good Characters,Living Characters,2258.0,Blue Eyes,Nov-50,NaN,Blond Hair,No Dual Identity,Male Characters,NaN,1950.0,Thor (Thor Odinson),2460,\/Thor_(Thor_Odinson)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Female Characters,2006.0,NaN,Nadine West (New Earth),66302,\/wiki\/Nadine_West_(New_Earth)
6892,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,Warren Harding (New Earth),283475,\/wiki\/Warren_Harding_(New_Earth)
6893,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,William Harrison (New Earth),283478,\/wiki\/William_Harrison_(New_Earth)
6894,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,William McKinley (New Earth),283471,\/wiki\/William_McKinley_(New_Earth)


In [17]:
new_df.tail()

,ALIGN,ALIVE,APPEARANCES,EYE,FIRST APPEARANCE,GSM,HAIR,ID,SEX,YEAR,Year,name,page_id,urlslug
6891,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Female Characters,2006.0,NaN,Nadine West (New Earth),66302,\/wiki\/Nadine_West_(New_Earth)
6892,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,Warren Harding (New Earth),283475,\/wiki\/Warren_Harding_(New_Earth)
6893,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,William Harrison (New Earth),283478,\/wiki\/William_Harrison_(New_Earth)
6894,Good Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Black Hair,Public Identity,Male Characters,2006.0,NaN,William McKinley (New Earth),283471,\/wiki\/William_McKinley_(New_Earth)
6895,Bad Characters,Living Characters,1.0,Blue Eyes,"2010, December",NaN,Blond Hair,Public Identity,Male Characters,2006.0,NaN,Mookie (New Earth),150660,\/wiki\/Mookie_(New_Earth)


## Create Dummies
Use an encoder

In [19]:
new_df.head()

,ALIGN,ALIVE,APPEARANCES,EYE,FIRST APPEARANCE,GSM,HAIR,ID,SEX,YEAR,Year,name,page_id,urlslug
0,Good Characters,Living Characters,4043.0,Hazel Eyes,Aug-62,NaN,Brown Hair,Secret Identity,Male Characters,NaN,1962.0,Spider-Man (Peter Parker),1678,\/Spider-Man_(Peter_Parker)
1,Good Characters,Living Characters,3360.0,Blue Eyes,Mar-41,NaN,White Hair,Public Identity,Male Characters,NaN,1941.0,Captain America (Steven Rogers),7139,\/Captain_America_(Steven_Rogers)
2,Neutral Characters,Living Characters,3061.0,Blue Eyes,Oct-74,NaN,Black Hair,Public Identity,Male Characters,NaN,1974.0,"Wolverine (James \""Logan\"" Howlett)",64786,\/Wolverine_(James_%22Logan%22_Howlett)
3,Good Characters,Living Characters,2961.0,Blue Eyes,Mar-63,NaN,Black Hair,Public Identity,Male Characters,NaN,1963.0,"Iron Man (Anthony \""Tony\"" Stark)",1868,\/Iron_Man_(Anthony_%22Tony%22_Stark)
4,Good Characters,Living Characters,2258.0,Blue Eyes,Nov-50,NaN,Blond Hair,No Dual Identity,Male Characters,NaN,1950.0,Thor (Thor Odinson),2460,\/Thor_(Thor_Odinson)


In [58]:
# Create dummies separately for 'X' and 'Y'
#new_df = pd.get_dummies(new_df['EYE','HAIR','ID','SEX','ALIVE'])

new_df.info()
#new_df = pd.get_dummies(new_df['EYE'])
new_df = pd.get_dummies(new_df, columns = ['EYE','HAIR','SEX','ALIVE','ID'])

new_target_df= pd.get_dummies(new_df['ALIGN'])
#need drop original "EYE"
new_df.info()
new_df.head()
new_target_df.info()
new_target_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23272 entries, 0 to 6895
Data columns (total 14 columns):
ALIGN               23272 non-null object
ALIVE               23272 non-null object
APPEARANCES         23272 non-null float64
EYE                 23272 non-null object
FIRST APPEARANCE    23272 non-null object
GSM                 154 non-null object
HAIR                23272 non-null object
ID                  23272 non-null object
SEX                 23272 non-null object
YEAR                6896 non-null float64
Year                16376 non-null float64
name                23272 non-null object
page_id             23272 non-null int64
urlslug             23272 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23272 entries, 0 to 6895
Data columns (total 76 columns):
ALIGN                               23272 non-null object
APPEARANCES                         23272 non-null float64
FIRST APPEARANCE 

,Bad Characters,Good Characters,Neutral Characters,Reformed Criminals
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0


In [ ]:
#Should I convert target variable as string to float for feature selection? 

In [79]:
#characters in their comics, whereas DC has more number of reformed criminals in their comics. although they have similar number of bad charaters in their comics.
# for interactive visualizations
!pip install plotly==4.8.1

import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff
from bubbly.bubbly import bubbleplot

hair_dc = dc_hero_df['HAIR'].value_counts()
hair_marvel = marvel_hero_df['HAIR'].value_counts()

trace = go.Bar(
             x = hair_dc.index,
             y = hair_dc.values,
             name = 'DC',
             marker = dict(
                  color = 'rgb(56, 54, 36)'
             )
)
trace2 = go.Bar(
            x = hair_marvel.index,
            y = hair_marvel.values,
            name = 'Marvel',
            marker = dict(
                  color = 'rgb(78, 03, 45)'
            )
)

data = [trace, trace2]

layout = go.Layout(
             barmode = 'relative',
              title = 'Different Hair Colors of SuperHeroes')

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

     |████████████████████████████████| 11.5MB 4.1MB/s eta 0:00:01    |██▎                             | 819kB 1.1MB/s eta 0:00:10     |███████████████████             | 6.9MB 3.9MB/s eta 0:00:02     |██████████████████████▋         | 8.1MB 3.9MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11429 sha256=dac9b62db1a95010bd081fdd6c57157e9b63e9a43915c1bb7175fac7177d67f3
  Stored in directory: /Users/mehrunisaqayyum/Library/Caches/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


ModuleNotFoundError: No module named 'bubbly'

### Feature Selection
Source: https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e

In [75]:
#apply SelectKBest class to extract top 5 best features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestfeatures = SelectKBest(score_func=chi2, k=5)
fit = bestfeatures.fit(X,Y)

ValueError: could not convert string to float: 'Good Characters'

### PCA

In [14]:
X = merged_df.iloc[:,2:20]
print(X.shape)

(44469272, 18)


In [ ]:
### Normalize Data so that all variables have a mean of 0 and standard deviation
# of 1.
X = StandardScaler().fit_transform(new_df)

## Perceptron Model

In [60]:
# Need to Import Perceptron.
from sklearn.linear_model import Perceptron

# Establish X and Y.
X = new_df

Y = new_target_df

In [61]:
X

,ALIGN,APPEARANCES,FIRST APPEARANCE,GSM,YEAR,Year,name,page_id,urlslug,EYE_Amber Eyes,...,SEX_Genderless Characters,SEX_Male Characters,SEX_Transgender Characters,ALIVE_Deceased Characters,ALIVE_Living Characters,ID_Identity Unknown,ID_Known to Authorities Identity,ID_No Dual Identity,ID_Public Identity,ID_Secret Identity
0,Good Characters,4043.0,Aug-62,NaN,NaN,1962.0,Spider-Man (Peter Parker),1678,\/Spider-Man_(Peter_Parker),0,...,0,1,0,0,1,0,0,0,0,1
1,Good Characters,3360.0,Mar-41,NaN,NaN,1941.0,Captain America (Steven Rogers),7139,\/Captain_America_(Steven_Rogers),0,...,0,1,0,0,1,0,0,0,1,0
2,Neutral Characters,3061.0,Oct-74,NaN,NaN,1974.0,"Wolverine (James \""Logan\"" Howlett)",64786,\/Wolverine_(James_%22Logan%22_Howlett),0,...,0,1,0,0,1,0,0,0,1,0
3,Good Characters,2961.0,Mar-63,NaN,NaN,1963.0,"Iron Man (Anthony \""Tony\"" Stark)",1868,\/Iron_Man_(Anthony_%22Tony%22_Stark),0,...,0,1,0,0,1,0,0,0,1,0
4,Good Characters,2258.0,Nov-50,NaN,NaN,1950.0,Thor (Thor Odinson),2460,\/Thor_(Thor_Odinson),0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,Good Characters,1.0,"2010, December",NaN,2006.0,NaN,Nadine West (New Earth),66302,\/wiki\/Nadine_West_(New_Earth),0,...,0,0,0,0,1,0,0,0,1,0
6892,Good Characters,1.0,"2010, December",NaN,2006.0,NaN,Warren Harding (New Earth),283475,\/wiki\/Warren_Harding_(New_Earth),0,...,0,1,0,0,1,0,0,0,1,0
6893,Good Characters,1.0,"2010, December",NaN,2006.0,NaN,William Harrison (New Earth),283478,\/wiki\/William_Harrison_(New_Earth),0,...,0,1,0,0,1,0,0,0,1,0
6894,Good Characters,1.0,"2010, December",NaN,2006.0,NaN,William McKinley (New Earth),283471,\/wiki\/William_McKinley_(New_Earth),0,...,0,1,0,0,1,0,0,0,1,0


In [62]:
Y

,Bad Characters,Good Characters,Neutral Characters,Reformed Criminals
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
6891,0,1,0,0
6892,0,1,0,0
6893,0,1,0,0
6894,0,1,0,0


## Split data to train and test

In [64]:
#Split data to train and test with 20% sample 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.25,random_state =5)

In [80]:
# Establish Perceptron Model.
# 10,000 iterations to ensure accuracy since data is non-normalized.
#perceptron = Perceptron(n_iter=10000)
### If running in your own environment on scikit-learn 0.21, run the line of code below instead:
perceptron = Perceptron(max_iter=10000, tol=0, n_iter_no_change=10000)

# Fit Perceptron.
perceptron.fit(X_train, Y_train)

# Get Parameters.
print('Score: ' + str(perceptron.score(X_train, Y_train)))

ValueError: could not convert string to float: '\\/Leslie_Lenrow_(Earth-616)'

## Random Forest Classifier Model

In [71]:
from sklearn.model_selection import cross_val_score

from sklearn import ensemble

rfc = ensemble.RandomForestClassifier()

X = new_df
Y = new_target_df
#X = pd.get_dummies(X)
#X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 304, in fit
    accept_sparse="csc", dtype=DTYPE)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 803, in check_X_y
    estimator=estimator)
  File "/opt/anaconda3/lib/python3.7/site-package

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 304, in fit
    accept_sparse="csc", dtype=DTYPE)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 803, in check_X_y
    estimator=estimator)
  File "/opt/anaconda3/lib/python3.7/site-package

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])